In [1]:
import cv2
from pathlib import Path

use_video = True
path = "/home/venky/co-tracker/videos/lab.mov"
pixel_key = "pixels"

# TODO: Set the task name here -- this will be used to save the output
task_name = "gripper_test"

img = None
if use_video:
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = frame.copy()
        break
    cap.release()
else:
    img = cv2.imread(path)

# We flip here because CV2 reads in as BGR
img = img[:, :, ::-1]

# Resize the image to 256x256
img = cv2.resize(img, (256, 256))

# TODO: If it's hard to see the image, you can increase the size_multiplier, this won't affect the selected coordinates
size_multiplier = 1

coordinates_path = f"../coordinates"

In [2]:
%gui asyncio

import os
from PIL import Image

import numpy as np
from IPython.display import display, Javascript
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
import pickle

import io
import asyncio
import logging

# Define an async function to wait for button click
async def wait_for_click(button):
    # Create a future object
    future = asyncio.Future()
    # Define the click event handler
    def on_button_clicked(b):
        future.set_result(None)
    # Attach the event handler to the button
    button.on_click(on_button_clicked)
    # Wait until the future is set
    await future

class Points():
    def __init__(self, env_name, img, coordinates_path, size_multiplier=1):
        logging.getLogger().setLevel(logging.DEBUG)
        logging.info("Starting the Points class")
        self.img = img
        self.size_multiplier = size_multiplier
        self.coordinates_path = coordinates_path
        self.env_name = env_name

        # Save the image to a bytes buffer
        image = Image.fromarray(self.img)
        size = img.shape
        image = image.resize((size[1] * self.size_multiplier, size[0] * self.size_multiplier))
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        buffer.seek(0)

        # Create an IPyWidgets Image widget
        self.canvas = Canvas(width=size[1] * self.size_multiplier, height=size[0] * self.size_multiplier)
        # Define the size of each cell

        self.canvas.put_image_data(np.array(image), 0, 0)

        # Display coordinates
        coords_label = widgets.Label(value="Click on the image to select the coordinates")

        # Define the click event handler
        self.coords = []
        def on_click(x, y):
            coords_label.value = f"Coordinates: ({x}, {y})"
            self.coords.append((0, x, y))

            with hold_canvas(self.canvas):
                self.canvas.put_image_data(np.array(image), 0, 0)  # Redraw the original image

                self.canvas.fill_style = 'red'
                for coord in self.coords:
                    x, y = coord[1] // self.size_multiplier, coord[2] // self.size_multiplier
                    self.canvas.fill_circle(x, y, 2)

        # Connect the click event to the handler
        self.canvas.on_mouse_down(on_click)

        self.button = widgets.Button(description="Save Points")

        # Display the widgets
        self.vbox = widgets.VBox([self.canvas, coords_label, self.button])

        # # Display the widget
        display(self.vbox)

    def on_done(self):
        logging.info("saving")
        Path(self.coordinates_path + "/coords/").mkdir(parents=True, exist_ok=True)
        with open(self.coordinates_path + "/coords/" + self.env_name + ".pkl", 'wb') as f:
            try:
                pickle.dump(self.coords, f)
            except Exception as e:
                logging.info(e)
        Path(self.coordinates_path + "/images/").mkdir(parents=True, exist_ok=True)
        with open(self.coordinates_path + "/images/" + self.env_name + ".png", 'wb') as f:
            try:
                image = Image.fromarray(self.img)
                image.save(f)
            except Exception as e:
                logging.info(e)
        logging.info("saved")

In [3]:
async def f():
    point = Points(task_name, img, coordinates_path, size_multiplier)
    x = await wait_for_click(point.button)
    point.vbox.close()
    point.canvas.close()
    point.on_done()
asyncio.ensure_future(f())

<Task pending name='Task-5' coro=<f() running at /tmp/ipykernel_80621/3187731821.py:1>>

INFO:root:Starting the Points class


DEBUG:Comm:handle_msg[1dbe1b0e5a2f4cef923ca5a582828e76]({'header': {'date': datetime.datetime(2025, 1, 1, 14, 48, 43, 427000, tzinfo=tzutc()), 'msg_id': '2ec0b64b-e8df-4bc0-8f52-8d59514c3ede', 'msg_type': 'comm_msg', 'session': '0b9692f6-9458-4cb8-a01c-0fae1580c0c6', 'username': 'ca3693f5-9d6b-48c1-a299-2c4bb5657705', 'version': '5.2'}, 'msg_id': '2ec0b64b-e8df-4bc0-8f52-8d59514c3ede', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': '1dbe1b0e5a2f4cef923ca5a582828e76', 'data': {'method': 'custom', 'content': {'event': 'client_ready'}}}, 'buffers': []})
DEBUG:Comm:handle_msg[1dbe1b0e5a2f4cef923ca5a582828e76]({'header': {'date': datetime.datetime(2025, 1, 1, 14, 48, 44, 553000, tzinfo=tzutc()), 'msg_id': 'eeb86895-75b3-41be-9e19-671734791cb5', 'msg_type': 'comm_msg', 'session': '0b9692f6-9458-4cb8-a01c-0fae1580c0c6', 'username': 'ca3693f5-9d6b-48c1-a299-2c4bb5657705', 'version': '5.2'}, 'msg_id': 'eeb86895-75b3-41be-9e19-671734791cb5', 'msg_type': 'comm

In [4]:
pkl_path = f"{coordinates_path}/coords/{task_name}.pkl"
pickle.load(open(pkl_path, 'rb'))

[(0, 35.58909809121989, 155.0020448216525),
 (0, 36.09224273726037, 159.36263175400336),
 (0, 28.041928400612623, 166.23894191655666),
 (0, 35.92452785524688, 209.8448112400653),
 (0, 79.02725253271502, 254.6246847376684),
 (0, 220.2431831880776, 155.84061923171998),
 (0, 218.39831948592916, 159.36263175400336),
 (0, 227.45492311465787, 169.42552467481306),
 (0, 219.40460877801013, 214.70854281845664),
 (0, 177.64360315664993, 255.29554426572238)]